In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import calendar
import chardet
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]

print(rj_archivos)
print(clasa_archivos)
print(mornese_archivos)

['REPORTE_CORTES_BBVA_2024-09.xlsx', 'REPORTE_CORTES_BBVA_2024-10.xlsx', 'REPORTE_CORTES_BBVA_2024-11.xlsx']
['REPORTE_CORTES_BBVA_2024-09.txt', 'REPORTE_CORTES_BBVA_2024-10.txt', 'REPORTE_CORTES_BBVA_2024-11.txt']
['REPORTE_CORTES_BBVA_2024-09.xlsx', 'REPORTE_CORTES_BBVA_2024-10.xlsx', 'REPORTE_CORTES_BBVA_2024-11.xlsx']


In [19]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [20]:
def detectar_codificacion(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

In [31]:
input_path = 'txt/ARCHIVO_RESPUESTA_PARTICULARES_13_12_2024.txt'
output_path = 'txt/ARCHIVO_RESPUESTA_PARTICULARES_13_12_2024_ASCII.txt'

try:
    # Leer el contenido del archivo en UTF-8
    with open(input_path, 'r', encoding='utf-8', errors='ignore') as infile:
        content = infile.read()
    
    # Guardar el contenido en el nuevo archivo con codificación ASCII
    with open(output_path, 'w', encoding='ascii', errors='ignore') as outfile:
        outfile.write(content)
    
    print(f"Archivo convertido a ASCII: {output_path}")

except FileNotFoundError:
    print(f"El archivo '{input_path}' no fue encontrado.")
except Exception as e:
    print(f"Se produjo un error: {e}")

Archivo convertido a ASCII: txt/ARCHIVO_RESPUESTA_PARTICULARES_13_12_2024_ASCII.txt


In [43]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA' in df.columns:
        df['AGENCIA'] = df['AGENCIA'].str.upper().replace('ASESCOM RJ ABOGADOS', 'ASESCOM RJ')
    if 'TIPO_DE_MARCACIÓN' in df.columns:
        df.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
    if 'TIPO_DE_DISCADO' in df.columns:
        df.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)
    if 'PRIORIDAD_TELEFONO' not in df.columns:
        df['PRIORIDAD_TELEFONO'] = np.nan
    
    df['MARCA'] = df['MARCA'].str.strip().str.upper().replace(' ', '', regex=True)
    df['MARCA'] = np.where(df['MARCA'] == 'KST', 'KSTBC', df['MARCA'])
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df['FECHA_LLAMADA'] = pd.to_datetime(df['FECHA_LLAMADA'], errors='coerce')
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['TIPO_CONTACTO'] = df['TIPO_CONTACTO'].str.upper().replace('TERCERO', 'CONTACTO INDIRECTO').replace('CONTACTO EFECTIVO', 'CONTACTO DIRECTO').replace('EFECTIVO', 'CONTACTO DIRECTO')
    df['TRAMO'] = df['TRAMO'].str.strip().str.upper().replace(' ', '', regex=True).replace(['T1', 'T2', 'T3', 'T4'], 'NULL')
    df['TRAMO'] = df['TRAMO'].fillna('NULL')
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.strip().str.upper().replace(' ', '', regex=True)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].fillna('NULL')
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    
    df.sort_values(by='FECHA_LLAMADA', inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.drop(columns={'HORA_INICIO_LLAMADA'}, inplace=True)
    
    df = df[['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']]
    return df

In [44]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        ruta_completa = os.path.join(agencia_path, archivo)
        if archivo.endswith('.txt'):
            df = pd.read_csv(
                ruta_completa, 
                sep=',', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
        else:
            df = pd.read_excel(
                ruta_completa, 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}
                )
        
        df.columns = clean_columns(df.columns)
        dataframes.append(df)
        print(f'{archivo} leído correctamente')
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado = clean_dataframe(df_consolidado)
    
    df_consolidado.to_csv(
        os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split("/")[-1]}.csv'), 
        index=False, 
        sep='|'
        )
    return df_consolidado

In [45]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': (rj_path, rj_archivos),
        'CLASA': (clasa_path, clasa_archivos),
        #'MORNESE': (mornese_path, mornese_archivos)
    }
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

REPORTE_CORTES_BBVA_2024-09.txt leído correctamente
REPORTE_CORTES_BBVA_2024-10.txt leído correctamente
REPORTE_CORTES_BBVA_2024-11.txt leído correctamente
Consolidado CLASA: (1112769, 10)


In [29]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        #'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.csv',
        'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.csv',
        #'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.csv'
    }
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_csv(
                file_path, 
                sep='|', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.csv no existe.')

root.destroy()

Consolidado CLASA leído: (1112769, 11)


In [100]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'TIPO_CONTACTO', 'PRIORIDAD_TELEFONO', 
            'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'PDP']
    return df[cols]

df_clasa_test: pd.DataFrame = dataframes['CLASA'].copy()
df_clasa_test.sort_values(by='FECHA_LLAMADA', inplace=True)
df_clasa_test.reset_index(inplace=True, drop=True)

print(df_clasa_test.columns)
print(df_clasa_test.shape)
df_clasa_test.head(5)

Index(['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA',
       'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO'],
      dtype='object')
(1112769, 10)


,AGENCIA,COD_CENTRAL,CONTRATO,MARCA,FECHA_LLAMADA,TIPO_CONTACTO,TRAMO,PDP,TIPO_MARCADOR,PRIORIDAD_TELEFONO
0,CLASA,24667250,001101135002005456,KSTBC,2024-09-02,NO CONTACTADO,NULL,0,PROGRESIVO,0
1,CLASA,27147059,001107855004153902,KSTBC,2024-09-02,NO CONTACTADO,C2,0,PREDICTIVO,2
2,CLASA,29206692,001102869600102269,EXJ,2024-09-02,NO CONTACTADO,NULL,0,PROGRESIVO,0
3,CLASA,27894645,001104379601512509,KSTBC,2024-09-02,NO CONTACTADO,NULL,0,PROGRESIVO,0
4,CLASA,28527397,001108149601444162,KSTBC,2024-09-02,NO CONTACTADO,C2,0,PREDICTIVO,0


In [101]:
df_clasa_test['PERIODO'] = pd.to_datetime(df_clasa_test['FECHA_LLAMADA']).dt.strftime('%Y%m').astype(str)

fechas_unicas = df_clasa_test[['PERIODO', 'FECHA_LLAMADA']].drop_duplicates().sort_values(by='FECHA_LLAMADA')
orden_fechas = fechas_unicas.groupby('PERIODO').cumcount() + 1
fechas_unicas['DIA'] = orden_fechas
df_clasa_test = df_clasa_test.merge(fechas_unicas[['PERIODO', 'FECHA_LLAMADA', 'DIA']], on=['PERIODO', 'FECHA_LLAMADA'], how='left')
df_clasa_test.drop(columns={'FECHA_LLAMADA'}, inplace=True)

df_clasa_test['INTENSIDAD_TOTAL'] = 1
df_clasa_test['INTENSIDAD_DIRECTA'] = np.where(df_clasa_test['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
df_clasa_test['CONTACTO_DIRECTO'] = np.where(df_clasa_test['INTENSIDAD_DIRECTA'] != 0, 1, 0)

df_clasa_test.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_clasa_test.reset_index(drop=True, inplace=True)

df_clasa_test = df_clasa_test[df_clasa_test['PERIODO'] == '202411']
df_clasa_test = df_clasa_test[df_clasa_test['DIA'] <= 5]
df_clasa_test

,AGENCIA,COD_CENTRAL,CONTRATO,MARCA,TIPO_CONTACTO,TRAMO,PDP,TIPO_MARCADOR,PRIORIDAD_TELEFONO,PERIODO,DIA,INTENSIDAD_TOTAL,INTENSIDAD_DIRECTA,CONTACTO_DIRECTO
732329,CLASA,23350378,001106599600092872,BC,NO CONTACTADO,C1,0,MANUAL,0,202411,1,1,0,0
732330,CLASA,29678190,001101149600223809,BC,NO CONTACTADO,C1,0,MANUAL,0,202411,1,1,0,0
732331,CLASA,23350378,001106599600092872,BC,CONTACTO DIRECTO,C1,1,MANUAL,0,202411,1,1,1,1
732332,CLASA,28410371,001103609600275696,BC,NO CONTACTADO,C1,0,MANUAL,0,202411,1,1,0,0
732333,CLASA,28410371,001103609600275696,BC,NO CONTACTADO,C1,0,MANUAL,0,202411,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821652,CLASA,20251023,001101955001543702,KSTBC,NO CONTACTADO,NULL,0,PREDICTIVO,0,202411,5,1,0,0
821653,CLASA,15348313,001108022501660311,KSTBC,CONTACTO DIRECTO,NULL,0,PREDICTIVO,0,202411,5,1,1,1
821654,CLASA,28958917,001103449600041477,KSTBC,CONTACTO INDIRECTO,NULL,0,PREDICTIVO,0,202411,5,1,0,0
821655,CLASA,22743235,001101575001458546,KSTBC,NO CONTACTADO,NULL,0,PREDICTIVO,2,202411,5,1,0,0


In [ ]:
# asignacion
#df_asig = pd.read_excel('bases/asignacion/202411/base_asignacion_NOV24.xlsx', sheet_name='BASE', dtype={'CONTRATO': str, 'COD_CENTRAL': str})
#df_asig = df_asig[df_asig['AGENCIA'] == 'CLASA MORA']
#df_asig['KEY'] = 'ASIGNACION'
#df_asig.shape

In [645]:
#df_asig.rename(columns={
#    'CC': 'COD_CENTRAL', 
#    'CARTERA': 'MARCA', 
#}, inplace=True)

#df_asig = df_asig[['COD_CENTRAL', 'CONTRATO','MARCA', 'AGENCIA', 'KEY']]
#df_asig['COD_CENTRAL'] = df_asig['COD_CENTRAL'].astype('Int64').astype(str).str.zfill(8)

In [684]:
#df_cruce = df_asig.merge(df_clasa_test, on='CONTRATO', how='left', suffixes=('', '_CLASA'))

#df_cruce.drop(columns={
#    'AGENCIA_CLASA',
#    'COD_CENTRAL_CLASA', 
#    'MARCA_CLASA', 
#}, inplace=True)

#df_cruce['TIPO_CONTACTO'] = df_cruce['TIPO_CONTACTO'].fillna('SIN GESTION')
#df_cruce['TRAMO'] = df_cruce['TRAMO'].fillna('NULL')
#df_cruce['PDP'] = df_cruce['PDP'].fillna(0)
#df_cruce['TIPO_MARCADOR'] = df_cruce['TIPO_MARCADOR'].fillna('SIN GESTION')
#df_cruce['PRIORIDAD_TELEFONO'] = df_cruce['PRIORIDAD_TELEFONO'].fillna('NULL')
#df_cruce['PERIODO'] = '202411'
#df_cruce['DIA'] = df_cruce['DIA'].fillna('NULL')

In [ ]:
#df_cruce['INTENSIDAD_TOTAL'] = np.where(df_cruce['INTENSIDAD_TOTAL'] == 1, 1, 0)
#df_cruce['INTENSIDAD_DIRECTA'] = np.where(df_cruce['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
#df_cruce['CONTACTO_DIRECTO'] = np.where(df_cruce['INTENSIDAD_DIRECTA'] != 0, 1, 0)

#df_cruce.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
#df_cruce.reset_index(drop=True, inplace=True)
#df_cruce

In [102]:
df_clasa_test = sort_columns(df_clasa_test)
df_clasa_test

,AGENCIA,COD_CENTRAL,CONTRATO,PERIODO,DIA,MARCA,TRAMO,TIPO_CONTACTO,PRIORIDAD_TELEFONO,TIPO_MARCADOR,INTENSIDAD_TOTAL,INTENSIDAD_DIRECTA,CONTACTO_DIRECTO,PDP
732329,CLASA,23350378,001106599600092872,202411,1,BC,C1,NO CONTACTADO,0,MANUAL,1,0,0,0
732330,CLASA,29678190,001101149600223809,202411,1,BC,C1,NO CONTACTADO,0,MANUAL,1,0,0,0
732331,CLASA,23350378,001106599600092872,202411,1,BC,C1,CONTACTO DIRECTO,0,MANUAL,1,1,1,1
732332,CLASA,28410371,001103609600275696,202411,1,BC,C1,NO CONTACTADO,0,MANUAL,1,0,0,0
732333,CLASA,28410371,001103609600275696,202411,1,BC,C1,NO CONTACTADO,0,MANUAL,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821652,CLASA,20251023,001101955001543702,202411,5,KSTBC,NULL,NO CONTACTADO,0,PREDICTIVO,1,0,0,0
821653,CLASA,15348313,001108022501660311,202411,5,KSTBC,NULL,CONTACTO DIRECTO,0,PREDICTIVO,1,1,1,0
821654,CLASA,28958917,001103449600041477,202411,5,KSTBC,NULL,CONTACTO INDIRECTO,0,PREDICTIVO,1,0,0,0
821655,CLASA,22743235,001101575001458546,202411,5,KSTBC,NULL,NO CONTACTADO,2,PREDICTIVO,1,0,0,0


In [85]:
df_clasa_test.to_excel('bases/gestiones/CLASA/CLASA_TEST.xlsx', index=False)

In [103]:
df_clasa_test['DIA'].value_counts(dropna=False)

DIA
1    20769
5    19681
2    17635
3    16448
4    14795
Name: count, dtype: int64

In [104]:
dias_unicos = sorted(df_clasa_test['DIA'].unique())
dias_unicos

[1, 2, 3, 4, 5]

In [111]:
df_test_groupped_1 = pd.DataFrame()

for dia in dias_unicos:
    df_acumulado = df_clasa_test[df_clasa_test['DIA'] <= dia]
    print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
    
    df_grouped = df_acumulado.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'CONTACTO_DIRECTO': 'max', 
        'PDP': 'max'
    }).reset_index()
    
    df_grouped['TASA_CIERRE'] = np.where(
        (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
        1, 
        np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
            0, 
            np.nan
            )
        )
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    df_grouped['DIA'] = dia
    
    df_test_groupped_1 = pd.concat([df_test_groupped_1, df_grouped])

df_test_groupped_1.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_test_groupped_1.reset_index(drop=True, inplace=True)

Dia acumulado: 1 (20769, 14) [1]
Dia acumulado: 2 (38404, 14) [1 2]
Dia acumulado: 3 (54852, 14) [1 2 3]
Dia acumulado: 4 (69647, 14) [1 2 3 4]
Dia acumulado: 5 (89328, 14) [1 2 3 4 5]


In [112]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'PERIODO', 'DIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'TASA_CIERRE']
    return df[cols]

df_test_groupped_1 = sort_columns(df_test_groupped_1)
df_test_groupped_1

,AGENCIA,PERIODO,DIA,COD_CENTRAL,CONTRATO,MARCA,TRAMO,PRIORIDAD_TELEFONO,TIPO_MARCADOR,INTENSIDAD_TOTAL,INTENSIDAD_DIRECTA,CONTACTO_DIRECTO,TASA_CIERRE
0,CLASA,202411,1,12846398,001107919600092153,BC,C1,0,MANUAL,1,1,1,1.0
1,CLASA,202411,1,22436596,001100309600066642,BC,C1,0,MANUAL,1,0,0,NaN
2,CLASA,202411,1,23099067,001100235000248026,BC,C1,0,MANUAL,5,0,0,NaN
3,CLASA,202411,1,23142426,001108439600053125,BC,C1,0,MANUAL,1,0,0,NaN
4,CLASA,202411,1,23350378,001106599600092872,BC,C1,0,MANUAL,5,2,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118069,CLASA,202411,5,30788071,001104379601916324,KSTBC,NULL,0,PROGRESIVO,2,0,0,NaN
118070,CLASA,202411,5,31082597,001103105002822269,KSTBC,NULL,0,PROGRESIVO,1,0,0,NaN
118071,CLASA,202411,5,31082597,001103109600906537,KSTBC,NULL,0,PROGRESIVO,1,0,0,NaN
118072,CLASA,202411,5,31082597,001103109600914068,KSTBC,NULL,0,PROGRESIVO,1,0,0,NaN


In [107]:
df_test_groupped_1.to_excel('bases/gestiones/CLASA/RESUMEN_5_DIAS_CLASA.xlsx', index=False)

In [108]:
os.startfile('C://Users//miria//Documents//BBVA//GitHub//base-pagos//bases//gestiones//CLASA//RESUMEN_5_DIAS_CLASA.xlsx')

In [109]:
df_clasa_test.isnull().sum()

AGENCIA               0
COD_CENTRAL           0
CONTRATO              0
PERIODO               0
DIA                   0
MARCA                 0
TRAMO                 0
TIPO_CONTACTO         0
PRIORIDAD_TELEFONO    0
TIPO_MARCADOR         0
INTENSIDAD_TOTAL      0
INTENSIDAD_DIRECTA    0
CONTACTO_DIRECTO      0
PDP                   0
dtype: int64

In [91]:
df_clasa_test['DIA'].value_counts(dropna=False).sort_index()

DIA
1    20769
2    17635
3    16448
4    14795
5    19681
Name: count, dtype: int64

In [92]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'CONTACTO_DIRECTO', 'TASA_CIERRE']
    return df[cols]

In [93]:
dias_unicos = sorted(df_clasa_test['DIA'].unique())
dias_unicos

[1, 2, 3, 4, 5]

In [94]:
df_test_groupped = pd.DataFrame()

for dia in dias_unicos:
    df_acumulado = df_clasa_test[df_clasa_test['DIA'] <= dia]
    print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
    
    df_grouped = df_acumulado.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'CONTACTO_DIRECTO': 'max', 
        'PDP': 'max'
    }).reset_index()
    
    df_grouped['TASA_CIERRE'] = np.where(
        (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
        1, 
        np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
            0, 
            np.nan
            )
        )
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    df_grouped = df_grouped.groupby(['AGENCIA', 'PERIODO', 'MARCA', 'TRAMO', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
        'INTENSIDAD_TOTAL': 'mean', 
        'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
        'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
        'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    }).reset_index()
    
    df_grouped['INTENSIDAD_TOTAL'] = df_grouped['INTENSIDAD_TOTAL'].round(2)
    df_grouped['INTENSIDAD_DIRECTA'] = df_grouped['INTENSIDAD_DIRECTA'].round(2)
    df_grouped['CONTACTO_DIRECTO'] = df_grouped['CONTACTO_DIRECTO'].round(2)
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].round(2)
    df_grouped['DIA'] = dia
    
    df_test_groupped = pd.concat([df_test_groupped, df_grouped])

df_test_groupped.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_test_groupped.reset_index(drop=True, inplace=True)
df_test_groupped.shape

Dia acumulado: 1 (20769, 14) [1]
Dia acumulado: 2 (38404, 14) [1 2]
Dia acumulado: 3 (54852, 14) [1 2 3]
Dia acumulado: 4 (69647, 14) [1 2 3 4]
Dia acumulado: 5 (89328, 14) [1 2 3 4 5]


(771, 11)

In [95]:
df_test_groupped = sort_columns(df_test_groupped)
df_test_groupped

,AGENCIA,PERIODO,DIA,MARCA,TRAMO,PRIORIDAD_TELEFONO,TIPO_MARCADOR,INTENSIDAD_TOTAL,INTENSIDAD_DIRECTA,CONTACTO_DIRECTO,TASA_CIERRE
0,CLASA,202411,1,BC,C1,0,MANUAL,3.47,1.12,0.42,1.00
1,CLASA,202411,1,BC,C1,0,PROGRESIVO,2.91,NaN,0.00,NaN
2,CLASA,202411,1,BC,C1,1,MANUAL,3.00,1.00,0.26,1.00
3,CLASA,202411,1,BC,C1,1,PROGRESIVO,2.33,NaN,0.00,NaN
4,CLASA,202411,1,BC,C1,2,MANUAL,2.50,NaN,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
766,CLASA,202411,5,KSTBC,C3,5+,PROGRESIVO,2.00,NaN,0.00,NaN
767,CLASA,202411,5,KSTBC,NULL,0,MANUAL,1.72,1.31,0.26,0.62
768,CLASA,202411,5,KSTBC,NULL,0,PREDICTIVO,1.68,1.02,0.05,0.11
769,CLASA,202411,5,KSTBC,NULL,0,PROGRESIVO,2.84,NaN,0.00,NaN


In [96]:
df_test_groupped.isnull().sum()

AGENCIA                 0
PERIODO                 0
DIA                     0
MARCA                   0
TRAMO                   0
PRIORIDAD_TELEFONO      0
TIPO_MARCADOR           0
INTENSIDAD_TOTAL        0
INTENSIDAD_DIRECTA    233
CONTACTO_DIRECTO        0
TASA_CIERRE           233
dtype: int64

In [97]:
df_test_groupped.fillna(0, inplace=True)

In [98]:
df_test_groupped.isnull().sum()

AGENCIA               0
PERIODO               0
DIA                   0
MARCA                 0
TRAMO                 0
PRIORIDAD_TELEFONO    0
TIPO_MARCADOR         0
INTENSIDAD_TOTAL      0
INTENSIDAD_DIRECTA    0
CONTACTO_DIRECTO      0
TASA_CIERRE           0
dtype: int64

In [99]:
df_test_groupped['DIA'].value_counts(dropna=False).sort_index()

DIA
1    126
2    151
3    160
4    162
5    172
Name: count, dtype: int64

In [114]:
df_test_groupped.to_excel('bases/gestiones/CLASA/PRUEBA.xlsx', index=False)
os.startfile('C://Users//miria//Documents//BBVA//GitHub//base-pagos//bases//gestiones//CLASA//PRUEBA.xlsx')